<a href="https://colab.research.google.com/github/sharon220596/python/blob/main/hr_policy_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Install dependencies

In [18]:
!pip install -q langchain-google-genai google-generativeai
!pip install -q sentence-transformers

## Create reuqired files

In [19]:
hr_policy_text = """
Leave Policy:
- Annual Leave: 18 days per year. Carry forward up to 6 days.
- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.
- Casual Leave: 6 days per year.

Attendance Policy:
- Working hours: 9:30 AM to 6:30 PM.
- Minimum 8 working hours per day.
- 3 late arrivals per month allowed.

Remote Work Policy:
- Eligible after 6 months tenure.
- Max 2 days per week.
- Not allowed during notice period.

Holiday Policy:
- 10 fixed company holidays per year.

Code of Conduct:
- Professional behavior mandatory.
- No harassment or misuse of company resources.

Exit Policy:
- Notice period: 60 days.
- Buyout subject to HR approval.
"""



In [20]:
file_path = "/content/hr_policies.txt"

with open(file_path, "w") as f:
    f.write(hr_policy_text)

print("File created at:", file_path)


File created at: /content/hr_policies.txt


In [21]:
# Setup the API Key in to os.environ variables from Colab
from google.colab import userdata
import os

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

# Prepare Data

In [22]:
# Load the Document
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path)
documents = loader.load()

documents[0].page_content[:300]


'\nLeave Policy:\n- Annual Leave: 18 days per year. Carry forward up to 6 days.\n- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.\n- Casual Leave: 6 days per year.\n\nAttendance Policy:\n- Working hours: 9:30 AM to 6:30 PM.\n- Minimum 8 working hours per day.\n- 3 late arrivals per '

In [23]:
# Split Document into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)

chunks = text_splitter.split_documents(documents)
len(chunks)


11

In [25]:
# Create Embeddings & Vector Store (FAISS)

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


/tmp/ipython-input-2666089695.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Tools

In [26]:
# Define HR Tools
from langchain_core.tools import tool
from datetime import datetime

@tool
def classify_hr_policy(query: str) -> str:
    """Classify HR question into policy category."""
    q = query.lower()
    if "leave" in q:
        return "Leave Policy"
    if "remote" in q or "work from home" in q:
        return "Remote Work Policy"
    if "late" in q or "attendance" in q:
        return "Attendance Policy"
    if "notice" in q or "resign" in q:
        return "Exit Policy"
    return "General HR Policy"


@tool
def calculate_days(start_date: str, end_date: str) -> str:
    """Calculate number of days between two dates (YYYY-MM-DD)."""
    s = datetime.strptime(start_date, "%Y-%m-%d")
    e = datetime.strptime(end_date, "%Y-%m-%d")
    return f"{(e - s).days} days"



In [27]:
def get_hr_context(question: str) -> str:
    docs = retriever.invoke(question)
    return "\n\n".join(d.page_content for d in docs)


In [28]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

hr_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
You are an HR Policy Assistant.

Strict rules:
- Answer ONLY using HR policy context provided in system messages.
- Do NOT guess or invent policies.
- If information is insufficient, say: "Not defined in policy".
- If the question is ambiguous, ask a clarification question.
- Maintain a professional HR tone.

Always respond using this structure:

Answer:
Source Policy:
Notes:
Confidence Level:
"""),
    MessagesPlaceholder(variable_name="messages")
])


# RAG

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | ChatPromptTemplate.from_messages([
        ("system", "Use the following HR policy context:\n\n{context}"),
        ("human", "{question}")
    ])
    | llm
    | StrOutputParser()
)


In [45]:
# Create agent

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[classify_hr_policy, calculate_days],
    prompt=hr_prompt
)


/tmp/ipython-input-2917807842.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [46]:
def ask_hr(question: str):
    context = get_hr_context(question)

    messages = [
        {
            "role": "system",
            "content": f"HR Policy Context:\n{context}"
        },
        {
            "role": "user",
            "content": question
        }
    ]

    return agent.invoke({"messages": messages})


In [47]:
# Ask a Normal HR Question
result = ask_hr("How many annual leave days do I get?")

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)

HR Policy Context:
- Casual Leave: 6 days per year.

Leave Policy:
- Annual Leave: 18 days per year. Carry forward up to 6 days.

- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.
How many annual leave days do I get?
[{'type': 'text', 'text': 'Answer: You get 18 days of annual leave per year.\nSource Policy: Leave Policy\nNotes: None\nConfidence Level: High\nHR Policy Context:\n- Casual Leave: 6 days per year.\n\nLeave Policy:\n- Annual Leave: 18 days per year. Carry forward up to 6 days.\n\n- Sick Leave: 10 days per year. Medical certificate required beyond 2 days.', 'extras': {'signature': 'CusBAXLI2nwECT3CU0yDkZ++repVE2XqoIFrPEqoJp/bfXcgeMuseUJxSOWveCJCwK+zIZWejugCnPwDF5fpaiHocOoqLmI0/lGfrJkl+DbxM6n4RCx3sWfIuEXEq3NjLHYhxvndJU/2QyMChfwMLO+zhiFn0z/6QOkXB2y7pVVR4/S5QMiaPwQTYzziZaBSyHBcuQh37GqoOBazhKW83VnviKkm7IkDGZfQWumWNfOUJsyhLRXsgJw3PTgjzO1MwXZan4t9LUqtO2qfpKDQelv0qS3Dw5F/Xsr+JcFSCSB8xPuGL/9hrlpyg0i1Tg=='}}]


In [48]:
# Clarification Example
result = ask_hr("Can I apply for remote work?")

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)


HR Policy Context:
Remote Work Policy:
- Eligible after 6 months tenure.
- Max 2 days per week.

- No harassment or misuse of company resources.

Attendance Policy:
- Working hours: 9:30 AM to 6:30 PM.
- Minimum 8 working hours per day.
Can I apply for remote work?

Remote Work Policy
[{'type': 'text', 'text': 'Answer:\nYes, you can apply for remote work if you have completed 6 months of tenure. If approved, you will be eligible for a maximum of 2 days of remote work per week.\n\nSource Policy:\nRemote Work Policy\n\nNotes:\nThe policy specifies eligibility criteria for remote work but does not detail the application process.\n\nConfidence Level:\nHigh\n\nHR Policy Context:\nRemote Work Policy:\n- Eligible after 6 months tenure.\n- Max 2 days per week.\n\n- No harassment or misuse of company resources.\n\nAttendance Policy:\n- Working hours: 9:30 AM to 6:30 PM.\n- Minimum 8 working hours per day.', 'extras': {'signature': 'CoAGAXLI2ny/+X0rzaHh3Pk/HqUgK1pdo108CDmv56u0i9uLwjQK+7K7a9moiE3

In [ ]:
# Out-of-Scope Question
result = ask_hr("How is my salary calculated?")

for msg in result["messages"]:
    if hasattr(msg, "content"):
        print(msg.content)